Extracción de datos ( 001_extract.ipynb)
Este cuaderno tiene como objetivo la extracción de datos desde [origen de datos], asegurando que la información obtenida sea completa y estructurada correctamente para su posterior transformación y análisis en el pipeline ETL.

Setting the environment

In [6]:
import sys
import os
sys.path.append(os.path.abspath(os.getcwd()))
sys.path.append('../config')

Importing libraries and modules

In [7]:
from sqlalchemy import text
import pandas as pd

Reading the dataset

In [ ]:
# Cargar el CSV asegurando que el delimitador sea correcto
df = pd.read_csv(r"C:/Users/ASUS/Desktop/proyecto/datos/traffic_accidents.csv")
df

,crash_date,traffic_control_device,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,crash_type,...,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,crash_hour,crash_day_of_week,crash_month
0,07/29/2023 01:00:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,13,7,7
1,08/13/2023 12:11:00 AM,TRAFFIC SIGNAL,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0,1,8
2,12/09/2021 10:30:00 AM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,10,5,12
3,08/09/2023 07:55:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,...,NONINCAPACITATING INJURY,5.0,0.0,0.0,5.0,0.0,0.0,19,4,8
4,08/19/2023 02:55:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,14,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209301,09/13/2023 01:08:00 PM,UNKNOWN,UNKNOWN,UNKNOWN,TURNING,FOUR WAY,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,13,4,9
209302,07/18/2023 02:10:00 PM,UNKNOWN,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,14,3,7
209303,10/23/2019 01:32:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,PEDESTRIAN,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT ON GRADE,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,2.0,0.0,2.0,0.0,0.0,0.0,13,4,10
209304,06/01/2020 03:23:00 PM,NO CONTROLS,CLEAR,DAYLIGHT,PEDESTRIAN,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,...,NONINCAPACITATING INJURY,1.0,0.0,0.0,1.0,0.0,1.0,15,2,6


In [9]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from config.conexion_db import conectar_db


Connection with PostgreSQL and Table creation

In [6]:
from conexion_db import conectar_db


def create_table():
    conexion = conectar_db()
    cursor = conexion.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS accidentes (
            id SERIAL PRIMARY KEY,
            crash_date TIMESTAMP,
            traffic_control_device TEXT,
            weather_condition TEXT,
            lighting_condition TEXT,
            first_crash_type TEXT,
            trafficway_type TEXT,
            alignment TEXT,
            roadway_surface_cond TEXT,
            road_defect TEXT,
            crash_type TEXT,
            intersection_related CHAR(1),
            damage TEXT,
            prim_contributory_cause TEXT,
            num_units INT,
            most_severe_injury TEXT,
            injuries_total FLOAT,
            injuries_fatal FLOAT,
            injuries_incapacitating FLOAT,
            injuries_non_incapacitating FLOAT,
            injuries_reported_not_evident FLOAT,
            injuries_no_indication FLOAT,
            crash_hour INT,
            crash_day_of_week INT,
            crash_month INT
        );
    """)

    conexion.commit()
    print("Tabla 'accidentes' creada exitosamente.")

    cursor.close()
    conexion.close()


if __name__ == "__main__":
    create_table()

✅ conexion_db.py se está ejecutando correctamente
✅ Conexión exitosa con psycopg2
Tabla 'accidentes' creada exitosamente.


In [7]:
df['crash_date'] = pd.to_datetime(df['crash_date'], format='%m/%d/%Y %I:%M:%S %p')

# Convertir NaN a None para evitar problemas en PostgreSQL
df = df.where(pd.notna(df), None)

Transfering the data to the database in PostgreSQL

In [8]:
from conexion_db import conectar_db


conexion = conectar_db()
cursor = conexion.cursor()

query = """
    INSERT INTO accidentes (
        crash_date, traffic_control_device, weather_condition, lighting_condition,
        first_crash_type, trafficway_type, alignment, roadway_surface_cond,
        road_defect, crash_type, intersection_related, damage,
        prim_contributory_cause, num_units, most_severe_injury,
        injuries_total, injuries_fatal, injuries_incapacitating,
        injuries_non_incapacitating, injuries_reported_not_evident,
        injuries_no_indication, crash_hour, crash_day_of_week, crash_month
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Convertir el DataFrame en una lista de tuplas
data = df.values.tolist()

# Insertar los datos en la tabla
cursor.executemany(query, data)  

# Confirmar la transacción
conexion.commit()
print("Datos insertados exitosamente.")

# Cerrar la conexión
cursor.close()
conexion.close()


✅ Conexión exitosa con psycopg2
Datos insertados exitosamente.


Verifying the data transference

In [10]:
# Verificar los datos insertados
conexion = conectar_db()

df_verificacion = pd.read_sql("SELECT * FROM accidentes LIMIT 5;", conexion)
df_verificacion


✅ Conexión exitosa con psycopg2


C:\Users\ASUS\AppData\Local\Temp\ipykernel_13568\1011235367.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_verificacion = pd.read_sql("SELECT * FROM accidentes LIMIT 5;", conexion)


,id,crash_date,traffic_control_device,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,...,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,crash_hour,crash_day_of_week,crash_month
0,1,2023-07-29 13:00:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,13,7,7
1,2,2023-08-13 00:11:00,TRAFFIC SIGNAL,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0,1,8
2,3,2021-12-09 10:30:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,10,5,12
3,4,2023-08-09 19:55:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,NONINCAPACITATING INJURY,5.0,0.0,0.0,5.0,0.0,0.0,19,4,8
4,5,2023-08-19 14:55:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,14,7,8
